In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df = pd.read_csv("Zomato Restaurant names and Metadata.csv")

In [3]:
zf = pd.read_csv("Zomato Restaurant reviews.csv")

In [4]:
df.head()

,Name,Links,Cost,Collections,Cuisines,Timings
0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
1,Paradise,https://www.zomato.com/hyderabad/paradise-gach...,800,Hyderabad's Hottest,"Biryani, North Indian, Chinese",11 AM to 11 PM
2,Flechazo,https://www.zomato.com/hyderabad/flechazo-gach...,"1,300","Great Buffets, Hyderabad's Hottest","Asian, Mediterranean, North Indian, Desserts","11:30 AM to 4:30 PM, 6:30 PM to 11 PM"
3,Shah Ghouse Hotel & Restaurant,https://www.zomato.com/hyderabad/shah-ghouse-h...,800,Late Night Restaurants,"Biryani, North Indian, Chinese, Seafood, Bever...",12 Noon to 2 AM
4,Over The Moon Brew Company,https://www.zomato.com/hyderabad/over-the-moon...,"1,200","Best Bars & Pubs, Food Hygiene Rated Restauran...","Asian, Continental, North Indian, Chinese, Med...","12noon to 11pm (Mon, Tue, Wed, Thu, Sun), 12no..."


In [5]:
df.shape

(105, 6)

In [6]:
zf.shape

(10000, 7)

In [7]:
df.drop(['Collections'], axis=1, inplace=True)

In [8]:
df.dropna(subset=['Timings'],inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 0 to 104
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      104 non-null    object
 1   Links     104 non-null    object
 2   Cost      104 non-null    object
 3   Cuisines  104 non-null    object
 4   Timings   104 non-null    object
dtypes: object(5)
memory usage: 4.9+ KB


## -----------------------*********************************************************-----------------------

In [10]:
zf.head()

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0


In [11]:
zf.dropna(subset=['Restaurant', 'Reviewer', 'Review', 'Rating', 'Metadata', 'Time',
       'Pictures'],inplace=True)

In [12]:
zf.isnull().sum()

Restaurant    0
Reviewer      0
Review        0
Rating        0
Metadata      0
Time          0
Pictures      0
dtype: int64

In [13]:
zf['Review'].duplicated().sum()

591

In [14]:
zf.drop(['Time', 'Pictures'], axis=1, inplace=True)

In [15]:
zf

,Restaurant,Reviewer,Review,Rating,Metadata
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers"
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers"
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers"
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower"
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers"
...,...,...,...,...,...
9995,Chinese Pavilion,Abhishek Mahajan,Madhumathi Mahajan Well to start with nice cou...,3,"53 Reviews , 54 Followers"
9996,Chinese Pavilion,Sharad Agrawal,This place has never disappointed us.. The foo...,4.5,"2 Reviews , 53 Followers"
9997,Chinese Pavilion,Ramandeep,"Bad rating is mainly because of ""Chicken Bone ...",1.5,"65 Reviews , 423 Followers"
9998,Chinese Pavilion,Nayana Shanbhag,I personally love and prefer Chinese Food. Had...,4,"13 Reviews , 144 Followers"


In [16]:
#Renaming Colunms
zf.rename(columns = {"Restaurant" : 'resto_name', "Reviewer" : 'reviewer', "Review" : 'review', "Rating" : 'rating', "Metadata" : 'metadata'}, inplace=True)

In [17]:
data = zf.reset_index()

In [18]:
data.head()

,index,resto_name,reviewer,review,rating,metadata
0,0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers"
1,1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers"
2,2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers"
3,3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower"
4,4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers"


In [19]:
data['review'][56]

"We went for lunch buffet yesterday and took a chance after seeing some good and some average reviews in Zomato.\n\nThe spread in buffet is too less, just 3 veg and 3 non veg starters, 2 main course in veg and non veg, just one fried rice and chicken Biryani.\nDesserts are ice cream and jamun.\n\nThe taste was ok and not to expect too much. Little disappointing when starters are not hot.\n\nFish fry was decent, Schezwan chicken was ok.\nVeg spring rolls were too good.\nBig surprise for me is Onion rings as starters 🤣😂\n\nKadai ghost in main course was good but missing that masala flavour.\nChicken was ok. Biryani is average, egg fried rice was good.\n\nFor the money we pay, we expect some good food. They use much of colour in food item's.\n\nSuggestion to management is use less coating and less food colouring in starters.\n\nOverall experience is ok, but they can be much better for the amount you pay."

In [20]:
list1 =[]

In [21]:
#Removing Emojis
#!pip install emoji
import emoji
def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

In [22]:
for i in data['review']:
    j = remove_emoji(i)
    list1.append(j)

C:\Users\Wooiii\AppData\Local\Temp/ipykernel_20576/1898636359.py:5: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(u'', string)


In [23]:
list1

['The ambience was good, food was quite good . had Saturday lunch , which was cost effective .\nGood place for a sate brunch. One can also chill with friends and or parents.\nWaiter Soumen Das was really courteous and helpful.',
 'Ambience is too good for a pleasant evening. Service is very prompt. Food is good. Over all a good experience. Soumen Das - kudos to the service',
 'A must try.. great food great ambience. Thnx for the service by Pradeep and Subroto. My personal recommendation is Penne Alfredo Pasta:) ....... Also the music in the background is amazing.',
 'Soumen das and Arun was a great guy. Only because of their behavior and sincerety, And good food off course, I would like to visit this place again.',
 'Food is good.we ordered Kodi drumsticks and basket mutton biryani. All are good. Thanks to Pradeep. He served well. We enjoyed here. Ambience is also very good.',
 'Ambiance is good, service is good, food is aPradeecp and subro best service\nFood is good. Papiya good hoste

In [24]:
list1[56]

"We went for lunch buffet yesterday and took a chance after seeing some good and some average reviews in Zomato.\n\nThe spread in buffet is too less, just 3 veg and 3 non veg starters, 2 main course in veg and non veg, just one fried rice and chicken Biryani.\nDesserts are ice cream and jamun.\n\nThe taste was ok and not to expect too much. Little disappointing when starters are not hot.\n\nFish fry was decent, Schezwan chicken was ok.\nVeg spring rolls were too good.\nBig surprise for me is Onion rings as starters \n\nKadai ghost in main course was good but missing that masala flavour.\nChicken was ok. Biryani is average, egg fried rice was good.\n\nFor the money we pay, we expect some good food. They use much of colour in food item's.\n\nSuggestion to management is use less coating and less food colouring in starters.\n\nOverall experience is ok, but they can be much better for the amount you pay."

In [25]:
#Removing \n from reviews
list2 = []
for o in list1:
    b = o
    bb = b.replace("\n","")
    list2.append(bb)

In [26]:
list2[56]

"We went for lunch buffet yesterday and took a chance after seeing some good and some average reviews in Zomato.The spread in buffet is too less, just 3 veg and 3 non veg starters, 2 main course in veg and non veg, just one fried rice and chicken Biryani.Desserts are ice cream and jamun.The taste was ok and not to expect too much. Little disappointing when starters are not hot.Fish fry was decent, Schezwan chicken was ok.Veg spring rolls were too good.Big surprise for me is Onion rings as starters Kadai ghost in main course was good but missing that masala flavour.Chicken was ok. Biryani is average, egg fried rice was good.For the money we pay, we expect some good food. They use much of colour in food item's.Suggestion to management is use less coating and less food colouring in starters.Overall experience is ok, but they can be much better for the amount you pay."

In [27]:
#Adding new_review after removing emojis and \n
data["new_review"] = pd.Series(list2)

In [28]:
data.head()

,index,resto_name,reviewer,review,rating,metadata,new_review
0,0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers","The ambience was good, food was quite good . h..."
1,1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",Ambience is too good for a pleasant evening. S...
2,2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",A must try.. great food great ambience. Thnx f...
3,3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",Soumen das and Arun was a great guy. Only beca...
4,4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",Food is good.we ordered Kodi drumsticks and ba...


In [29]:
data['new_review'][56]

"We went for lunch buffet yesterday and took a chance after seeing some good and some average reviews in Zomato.The spread in buffet is too less, just 3 veg and 3 non veg starters, 2 main course in veg and non veg, just one fried rice and chicken Biryani.Desserts are ice cream and jamun.The taste was ok and not to expect too much. Little disappointing when starters are not hot.Fish fry was decent, Schezwan chicken was ok.Veg spring rolls were too good.Big surprise for me is Onion rings as starters Kadai ghost in main course was good but missing that masala flavour.Chicken was ok. Biryani is average, egg fried rice was good.For the money we pay, we expect some good food. They use much of colour in food item's.Suggestion to management is use less coating and less food colouring in starters.Overall experience is ok, but they can be much better for the amount you pay."

## ------------------------------*************************--------------------------------

In [30]:
#New DataFrame
df1 = pd.DataFrame()

In [31]:
df1['Name'] = pd.Series(df['Name'])

In [32]:
df1.head(5)

,Name
0,Beyond Flavours
1,Paradise
2,Flechazo
3,Shah Ghouse Hotel & Restaurant
4,Over The Moon Brew Company


In [33]:
df.head(5)

,Name,Links,Cost,Cuisines,Timings
0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
1,Paradise,https://www.zomato.com/hyderabad/paradise-gach...,800,"Biryani, North Indian, Chinese",11 AM to 11 PM
2,Flechazo,https://www.zomato.com/hyderabad/flechazo-gach...,"1,300","Asian, Mediterranean, North Indian, Desserts","11:30 AM to 4:30 PM, 6:30 PM to 11 PM"
3,Shah Ghouse Hotel & Restaurant,https://www.zomato.com/hyderabad/shah-ghouse-h...,800,"Biryani, North Indian, Chinese, Seafood, Bever...",12 Noon to 2 AM
4,Over The Moon Brew Company,https://www.zomato.com/hyderabad/over-the-moon...,"1,200","Asian, Continental, North Indian, Chinese, Med...","12noon to 11pm (Mon, Tue, Wed, Thu, Sun), 12no..."


In [34]:
#Splitting Cuisines column using delimiter (,)
df2 = df['Cuisines'].str.split(',',expand=True)

In [35]:
df2.head(5)

,0,1,2,3,4,5
0,Chinese,Continental,Kebab,European,South Indian,North Indian
1,Biryani,North Indian,Chinese,None,None,None
2,Asian,Mediterranean,North Indian,Desserts,None,None
3,Biryani,North Indian,Chinese,Seafood,Beverages,None
4,Asian,Continental,North Indian,Chinese,Mediterranean,None


In [36]:
#Concatenating above DataFrames
df3 = pd.concat([df1, df2], axis=1)

In [37]:
df3.head()

,Name,0,1,2,3,4,5
0,Beyond Flavours,Chinese,Continental,Kebab,European,South Indian,North Indian
1,Paradise,Biryani,North Indian,Chinese,None,None,None
2,Flechazo,Asian,Mediterranean,North Indian,Desserts,None,None
3,Shah Ghouse Hotel & Restaurant,Biryani,North Indian,Chinese,Seafood,Beverages,None
4,Over The Moon Brew Company,Asian,Continental,North Indian,Chinese,Mediterranean,None


In [38]:
#Removine Blank Spaces
df3[0] = df3[0].str.lstrip()
df3[1] = df3[1].str.lstrip()
df3[2] = df3[2].str.lstrip()
df3[3] = df3[3].str.lstrip()
df3[4] = df3[4].str.lstrip()
df3[5] = df3[5].str.lstrip()

In [39]:
#Final DataFrame
df3.head()

,Name,0,1,2,3,4,5
0,Beyond Flavours,Chinese,Continental,Kebab,European,South Indian,North Indian
1,Paradise,Biryani,North Indian,Chinese,None,None,None
2,Flechazo,Asian,Mediterranean,North Indian,Desserts,None,None
3,Shah Ghouse Hotel & Restaurant,Biryani,North Indian,Chinese,Seafood,Beverages,None
4,Over The Moon Brew Company,Asian,Continental,North Indian,Chinese,Mediterranean,None


## ----------------------------------******************************************-----------------------------------

In [40]:
dict2  = dict(zip(df3['Name'], zip(df3[0],df3[1],df3[2],df3[3],df3[4],df3[5])))


In [41]:
dict2

{'Beyond Flavours': ('Chinese',
  'Continental',
  'Kebab',
  'European',
  'South Indian',
  'North Indian'),
 'Paradise': ('Biryani', 'North Indian', 'Chinese', None, None, None),
 'Flechazo': ('Asian',
  'Mediterranean',
  'North Indian',
  'Desserts',
  None,
  None),
 'Shah Ghouse Hotel & Restaurant': ('Biryani',
  'North Indian',
  'Chinese',
  'Seafood',
  'Beverages',
  None),
 'Over The Moon Brew Company': ('Asian',
  'Continental',
  'North Indian',
  'Chinese',
  'Mediterranean',
  None),
 "The Fisherman's Wharf": ('Seafood',
  'Goan',
  'North Indian',
  'Continental',
  'Asian',
  None),
 'eat.fit': ('Healthy Food',
  'North Indian',
  'Continental',
  'South Indian',
  None,
  None),
 'Shah Ghouse Spl Shawarma': ('Lebanese', None, None, None, None, None),
 'Hyper Local': ('American',
  'Chinese',
  'North Indian',
  'Kebab',
  'South Indian',
  None),
 'Cream Stone': ('Ice Cream', 'Desserts', None, None, None, None),
 "Sardarji's Chaats & More": ('Street Food',
  'North I

In [62]:
#function to get recommendation
def get_recommendations(cuisine):
    #Creating dictionary from Dataframe
    dict1  = dict(zip(df3['Name'], zip(df3[0],df3[1],df3[2],df3[3],df3[4],df3[5])))
    
    list6 = []
    
    #Appending Restaurants Names in list6 using dictionary 
    for x, y in dict1.items():
        for yy in y:
            if yy == cuisine:
                list6.append(x)
    
    
    Resto_name = list6[0]
    print('list6 :' ,list6[0])
    
    #Creating New DataFrame with Restaurants Names present in List6
    new_data = data[data["resto_name"].str.contains('|'.join(list6))]
    #new_data = new_data.iloc[: , 1:]

    
   # print('New DAta: ',new_data)
    tfidf = TfidfVectorizer(stop_words='english')
    
    tfidf_matrix = tfidf.fit_transform(new_data['new_review'])
    
    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    indices = pd.Series(df3.index, index=df3["Name"]).drop_duplicates()
    #print('INDC: ',indices)
    #def get_recommendations(name, cosine_sim=cosine_sim):
    # Get the index of the Restaurants that matches the Restaurants Name
    idx = indices[Resto_name]
    #print('IDX: ',idx)
    # Get the pairwsie similarity scores of all Restaurants with that Restaurants
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the Restaurants based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[2:50]
    
    # Get the restaurant indices
    #resto_indices = [i[0] for i in sim_scores]

    # Return the top most similar restaurants
    #res = new_data['resto_name'].iloc[resto_indices]
    #print('RS: ',res)
    res = []
    for i in sim_scores:
        res.append(new_data.iloc[i[0]].resto_name)
    print('Resto Name: ',Resto_name)

 
    list4 = []
    for r in res:
        list4.append(r)
    #print('List4 : ',list4)
        
    list5 = []
    for rec in list4:
        if rec not in list5:
            list5.append(rec)
    #print('List5 : ',list5)
        
    for rr in list5:
        print(rr)
   


    
    return cosine_sim
        
   
    

In [63]:
cuisine_name = input("Your Favorite Cuisine is ")
print("----------------------------------*************************--------------------------------")
print("Following Are the RECOMMENDATION According to your Favorite Cuisine")
similarity = get_recommendations(cuisine_name)

Your Favorite Cuisine is Kebab
----------------------------------*************************--------------------------------
Following Are the RECOMMENDATION According to your Favorite Cuisine
list6 : Beyond Flavours
Resto Name:  Beyond Flavours
Beyond Flavours
Prism Club & Kitchen
Tandoori Food Works
Hyper Local
La La Land - Bar & Kitchen
Barbeque Nation


In [54]:
#new_df['Cuisines'].drop_duplicates(inplace=True)

In [55]:
#import pickle

In [56]:
#pickle.dump(n_df,open('resto_name.pkl','wb'))

In [57]:
#pickle.dump(similarity,open('similarity.pkl','wb'))

In [58]:
#pickle.dump(df3,open('df3.pkl','wb'))

In [59]:
#pickle.dump(data.to_dict,open('data_dict.pkl','wb'))